<a href="https://colab.research.google.com/github/kg4-ken1ro/blockchain_test_with_python/blob/main/blockchain_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import hashlib
import json
import time
from queue import Queue


In [2]:
class Block:
  def __init__(self, index, timestamp, nonce, previous_block_hash, merkle_root, transactions):
    self.index = index #インデックス
    self.timestamp = timestamp #タイムスタンプ
    self.nonce = nonce #ナンス
    self.previous_block_hash = previous_block_hash #前のブロックハッシュ値
    self.merkle_root = merkle_root #マークルート
    self.transactions = transactions #トランザクション値
    self.block_hash = self.calculate_hash() #ブロックハッシュ値

  def calculate_hash(self): #SHA256でハッシュ計算する関数
    block_string = json.dumps({
        'index': self.index, #インデックス
        'timestamp': self.timestamp, #タイムスタンプ
        'nonce': self.nonce, #ナンス
        'previous_block_hash': self.previous_block_hash, #前のブロックハッシュ値
        'merkle_root': self.merkle_root, #マークルルート
    }, sort_keys=True).encode()

    return hashlib.sha256(block_string).hexdigest()

  def calculate_merkle_root(self, transactions): #マークルルートを計算する関数
    #要素数が0の場合は、空文字のハッシュ値を返す
    if len(transactions) == 0:
      return hashlib.sha256(b'').hexdigest()

    #要素数が1の場合は、そのトランザクションのハッシュ値を返す
    if len (transactions) == 1:
      return hashlib.sha256(transactions[0].encode()).hexdigest()

    #要素数が奇数の場合は、最後の要素を複製して偶数にする
    if len(transactions) % 2 == 1:
      transactions.append(transactions[-1])

    #ハッシュ値を格納するリストを初期化
    hashes = []

    #2つのトランザクションを組み合わせてハッシュ値を計算し、リストに格納する
    for i in range(0, len(transactions), 2):
      transaction_pair = transactions[i] + transactions[i+1]
      hash_value = hashlib.sha256(transaction_pair.encode()).hexdigest()
      hashes.append(hash_value)

    return self.calculate_merkle_root(hashes) #再帰的にマークルルートを計算


In [3]:
class Blockchain:
  def create_genesis_block(self): #ジェネシスブロックを生成する関数
    genesis_block = Block(
        index=0,
        timestamp= time.time(),
        nonce=0,
        previous_block_hash='',
        merkle_root='',
        transactions=[],
    )

    return genesis_block.__dict__
  
  def add_transaction(self, transaction): #トランザクションを追加する関数
    self.transaction_pool.put(transaction)
  
  def mine_block(self): #マイニングする関数
    previous_block = self.chain[-1] #直前のブロックを取得

    #新しいブロックのデーターを設定
    new_block_index = previous_block['index'] + 1
    new_block_timestamp = time.time()
    new_block_nonce = 0
    new_block_previous_hash = previous_block['block_hash']
    new_block_merkle_root = ''
    new_block_transactions = list(self.transaction_pool.queue)

    while True:
      #新しいブロックを作成
      new_block = Block(
          index = new_block_index,
          timestamp =new_block_timestamp,
          nonce = new_block_nonce,
          previous_block_hash = new_block_previous_hash,
          merkle_root = new_block_merkle_root,
          transactions = new_block_transactions
      )
      new_block.merkle_root = new_block.calculate_merkle_root(new_block_transactions)
      new_block_hash = new_block.calculate_hash()

      #ハッシュ値が要求された難易度より小さい場合、ブロックを追加して終了
      if new_block_hash[:self.difficulty] == '0' * self.difficulty:
        self.chain.append(new_block.__dict__)
        self.transaction_pool = Queue()

        return new_block.__dict__

      #nonceを増やして再試行
      new_block_nonce += 1

  def __init__(self):
    self.chain = [self.create_genesis_block()] #チェーン
    self.transaction_pool = Queue() #トランザクションブール
    self.difficulty = 2 #難易度


In [4]:
blockchain = Blockchain()
blockchain.add_transaction("Alice")
print(blockchain.chain)

[{'index': 0, 'timestamp': 1686555658.8240125, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5'}]


In [5]:
blockchain.mine_block()

{'index': 1,
 'timestamp': 1686555658.837591,
 'nonce': 1026,
 'previous_block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5',
 'merkle_root': '3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043',
 'transactions': ['Alice'],
 'block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8'}

In [6]:
print(blockchain.chain)

[{'index': 0, 'timestamp': 1686555658.8240125, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5'}, {'index': 1, 'timestamp': 1686555658.837591, 'nonce': 1026, 'previous_block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5', 'merkle_root': '3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043', 'transactions': ['Alice'], 'block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8'}]


In [7]:
blockchain.add_transaction('Bob')
blockchain.add_transaction('Grace')
print(blockchain.chain)

[{'index': 0, 'timestamp': 1686555658.8240125, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5'}, {'index': 1, 'timestamp': 1686555658.837591, 'nonce': 1026, 'previous_block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5', 'merkle_root': '3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043', 'transactions': ['Alice'], 'block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8'}]


In [8]:
blockchain.mine_block()

{'index': 2,
 'timestamp': 1686555658.9606955,
 'nonce': 10,
 'previous_block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8',
 'merkle_root': '8eb45a82f786577388e204d82ffbedc61fcbfaa7fc31a8fae6e0c163e35d5dd5',
 'transactions': ['Bob', 'Grace'],
 'block_hash': '04e40af866ed3c269dbbdde896e10494cb034868edc154275e4973df6a474554'}

In [9]:
print(blockchain.chain)

[{'index': 0, 'timestamp': 1686555658.8240125, 'nonce': 0, 'previous_block_hash': '', 'merkle_root': '', 'transactions': [], 'block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5'}, {'index': 1, 'timestamp': 1686555658.837591, 'nonce': 1026, 'previous_block_hash': '222a3c63f3b389044284e811d3bd42773cb38d1fa7e4c622391cc16dfa5912b5', 'merkle_root': '3bc51062973c458d5a6f2d8d64a023246354ad7e064b1e4e009ec8a0699a3043', 'transactions': ['Alice'], 'block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8'}, {'index': 2, 'timestamp': 1686555658.9606955, 'nonce': 10, 'previous_block_hash': '158db00c64dbb9261949192021053570d5d5ecf24c0af2925651c2a7e132fcd8', 'merkle_root': '8eb45a82f786577388e204d82ffbedc61fcbfaa7fc31a8fae6e0c163e35d5dd5', 'transactions': ['Bob', 'Grace'], 'block_hash': '04e40af866ed3c269dbbdde896e10494cb034868edc154275e4973df6a474554'}]
